# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

ModuleNotFoundError: No module named 'scipy'

In [ ]:
data = pd.read_csv('./data/customer_product_sales.csv')

In [ ]:
data.head()

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [ ]:
## Paso 1 groupby (customer_id, product_name) >> sum(quantity)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [ ]:
## pivot pable-> df, values, index, columns + fillna a 0 y fillvalues a 0

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [ ]:
dist_calculation = pdist(X=users_ratings, metric='euclidean')

# Frame of reference

euclid_dist_norm = pd.DataFrame(1/(1 + squareform(pdist(users_ratings, 'euclidean'))),
                                index=movie_ratings.columns,
                                columns=movie_ratings.columns)

euclid_dist_norm

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [ ]:
# Filter df(customer_id)
# Sort 
# head(5)

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [ ]:
# df(step1) + df(step4 - customerid) >> merge / join  >> df_final(customerid, customer_name....)

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
# groupby (productName) >> sum(productname + quantity) >> sort (ascending=False)

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [ ]:
# filter (records == 0) >> sort 5

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [5]:
# Pide una lista que contenga diccionarios [{}, {}, {}] >> para completarla bucle for

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.